In [7]:
import os
import numpy as np
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, SpectralClustering                                         
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score         
import matplotlib.pyplot as plt                                  
import cv2
from collections import Counter
from sklearn.mixture import GaussianMixture

In [8]:
def load_ground_truth(gt_path):                                     
    ground_truth = np.genfromtxt(gt_path, dtype=int, delimiter=',') 
    return ground_truth

def load_extracted_features(exp_path, folder):                      
    emb_save_path = os.path.join(exp_path, f'{folder}.npy')         
    features = np.load(emb_save_path, allow_pickle=True)                          
    return features                                                 
                                                                    
def perform_clustering(features, n_clusters):                     
    kmeans = KMeans(n_clusters=100, random_state=0).fit(features)   
    return kmeans.labels_                                           
def compare_with_ground_truth(pred_labels, true_labels):
    ari = adjusted_rand_score(true_labels, pred_labels)
    nmi = normalized_mutual_info_score(true_labels, pred_labels)
    return ari, nmi

In [22]:
gt_paths = [r'C:\Users\jleus\PycharmProjects\EE445\finalProj\EE443_2024_Challenge\data\val\camera_0005.txt', r'C:\Users\jleus\PycharmProjects\EE445\finalProj\EE443_2024_Challenge\data\val\camera_0017.txt', r'C:\Users\jleus\PycharmProjects\EE445\finalProj\EE443_2024_Challenge\data\val\camera_0025.txt']
detection_paths = [r'C:\Users\jleus\PycharmProjects\EE445\finalProj\EE443_2024_Challenge\runs\detect\inference\txt\camera_0005.txt', r'C:\Users\jleus\PycharmProjects\EE445\finalProj\EE443_2024_Challenge\runs\detect\inference\txt\camera_0017.txt', r'C:\Users\jleus\PycharmProjects\EE445\finalProj\EE443_2024_Challenge\runs\detect\inference\txt\camera_0025.txt']
exp_path = 'C:/Users/jleus/PycharmProjects/EE445/finalProj/EE443_2024_Challenge/runs/reid/inference'
folders = ['camera_0005', 'camera_0017', 'camera_0025']

In [23]:
ground_truths = [load_ground_truth(gt_path) for gt_path in gt_paths]
detect_inferences = [load_ground_truth(detect_path) for detect_path in detection_paths]

In [6]:
featuresAll = []
predictions = []
for i, folder in enumerate(folders): 
    print(f'Processing folder {folder}')
    features = load_extracted_features(exp_path, folder)
    features = np.vstack(features)
    # n_clusters = len(np.unique(ground_truths[i]))
    # pred_labels = perform_clustering(features, n_clusters)
    featuresAll.append(features)
    # ari, nmi = compare_with_ground_truth(pred_labels, ground_truths[i])

Processing folder camera_0005
Processing folder camera_0017
Processing folder camera_0025


In [43]:
column_data = ground_truths[0][:, 1]

In [44]:
len(np.unique(column_data))

25

In [151]:
kmeans = KMeans(n_clusters=25, random_state=0).fit(featuresAll[0])

In [45]:
agglo = AgglomerativeClustering(n_clusters=25).fit(featuresAll[0])

In [245]:
spect = SpectralClustering(n_clusters=24).fit(featuresAll[2])

In [280]:
gaus = GaussianMixture(n_components=25).fit(featuresAll[0][:500])

In [46]:
agglo.labels_

array([11,  1, 11, ..., 10,  3,  2], dtype=int64)

In [47]:
detect_inferences[0][:, 1] = agglo.labels_

In [48]:
detect_inferences[0]

array([[   5,   11,    0, ...,  163,    0,   -1],
       [   5,    1,    0, ...,  194,    0,   -1],
       [   5,   11,    0, ...,  160,    0,   -1],
       ...,
       [   5,   10, 1799, ...,  129,    0,   -1],
       [   5,    3, 1799, ...,  223,    0,   -1],
       [   5,    2, 1799, ...,  140,    0,   -1]])

In [24]:
def draw_boxes_on_frame(frame, detections): 
    for detection in detections: 
        camera_id, tracking_id, frame_id, xmin, ymin, width, height = detection[:7]
        x_center, y_center, width, height = xmin, ymin, width, height
        xmin = int(x_center - width / 2)
        ymin = int(y_center - height / 2)
        xmax = int(x_center + width / 2)
        ymax = int(y_center + height / 2)
        
        cv2.rectangle(frame, (xmin, ymin), (xmin + width, ymin + height), (0, 255, 0), 2)
        cv2.putText(frame, str(tracking_id), (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

In [49]:
# def linear_interpolation(prev_pos, curr_pos, alpha):
#     return prev_pos * (1 - alpha) + curr_pos * alpha
# def draw_boxes_on_frame(frame, detections, prev_positions, frame_id, alpha=0.1):
#     new_positions = []
#     for detection in detections:
#         camera_id, tracking_id, frame_id, xmin, ymin, width, height = detection[:7]
#         x_center, y_center = xmin, ymin
# 
#         if tracking_id in prev_positions:
#             prev_x_center, prev_y_center, prev_width, prev_height = prev_positions[tracking_id]
#             x_center = linear_interpolation(prev_x_center, x_center, alpha)
#             y_center = linear_interpolation(prev_y_center, y_center, alpha)
#             width = linear_interpolation(prev_width, width, alpha)
#             height = linear_interpolation(prev_height, height, alpha)
# 
#         new_positions.append((tracking_id, x_center, y_center, width, height))
# 
#         xmin = int(x_center - width / 2)
#         ymin = int(y_center - height / 2)
#         xmax = int(x_center + width / 2)
#         ymax = int(y_center + height / 2)
# 
#         cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
#         cv2.putText(frame, str(tracking_id), (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
# 
#     return {tracking_id: (x_center, y_center, width, height) for tracking_id, x_center, y_center, width, height in new_positions}

In [25]:
frame_directory = r'C:\Users\jleus\PycharmProjects\EE445\finalProj\EE443_2024_Challenge\data\val\camera_0005'
output_directory = r'C:\Users\jleus\PycharmProjects\EE445\finalProj\EE443_2024_Challenge\runs\tracking\vis\AgglomerativeClustering\camera_0005\images'

In [26]:
detect_tracking = load_ground_truth(r'C:\Users\jleus\PycharmProjects\EE445\finalProj\EE443_2024_Challenge\runs\tracking\inference\camera_0005.txt')

In [27]:
detect_tracking

array([[   5,    8,    0, ...,  163,   -1,   -1],
       [   5,    8,    1, ...,  161,   -1,   -1],
       [   5,    8,    2, ...,  161,   -1,   -1],
       ...,
       [   5,   15, 1797, ...,  143,   -1,   -1],
       [   5,   15, 1798, ...,  149,   -1,   -1],
       [   5,   15, 1799, ...,  155,   -1,   -1]])

In [28]:
detections_by_frame = {}
for detection in detect_tracking:
    frame_id = detection[2]
    if frame_id not in detections_by_frame: 
        detections_by_frame[frame_id] = []
    detections_by_frame[frame_id].append(detection)

In [29]:
for frame_id, detections in detections_by_frame.items(): 
    frame_path = f"{frame_directory}/{str(frame_id).zfill(5)}.jpg"
    frame = cv2.imread(frame_path)
    if frame is not None: 
        draw_boxes_on_frame(frame, detections)
        output_path = f"{output_directory}/annotated_{str(frame_id).zfill(5)}.jpg"
        cv2.imwrite(output_path, frame)
        
cv2.destroyAllWindows()

In [56]:
# prev_positions = {}
# for frame_id, detections in sorted(detections_by_frame.items()):
#     frame_path = f"{frame_directory}/{str(frame_id).zfill(5)}.jpg"
#     frame = cv2.imread(frame_path)
#     if frame is not None:
#         prev_positions = draw_boxes_on_frame(frame, detections, prev_positions, frame_id)
#         output_path = f"{output_directory}/annotated_{str(frame_id).zfill(5)}.jpg"
#         cv2.imwrite(output_path, frame)

In [30]:
first_frame_id = str(min(detections_by_frame.keys())).zfill(5)
first_frame_path = os.path.join(frame_directory, f"{first_frame_id}.jpg")
first_frame = cv2.imread(first_frame_path)
height, width, layers = first_frame.shape
video_path = os.path.join(r'C:\Users\jleus\PycharmProjects\EE445\finalProj\EE443_2024_Challenge\runs\tracking\vis\AgglomerativeClustering\camera_0005\camera_0005.mp4')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(video_path, fourcc, 20.0, (width, height))

In [31]:
for frame_id in sorted(detections_by_frame.keys()):
    frame_path = os.path.join(frame_directory, f"{str(frame_id).zfill(5)}.jpg")
    frame = cv2.imread(frame_path)
    if frame is not None:
        # Draw bounding boxes and tracking IDs
        draw_boxes_on_frame(frame, detections_by_frame[frame_id])
        # Write the frame into the video
        out.write(frame)
        # To display the frame, uncomment the lines below
        # cv2.imshow('Annotated Frame', frame)
        # cv2.waitKey(1)

# Release everything if job is finished
out.release()
cv2.destroyAllWindows()

In [58]:
# prev_positions = {}
# for frame_id in sorted(detections_by_frame.keys()):
#     frame_path = os.path.join(frame_directory, f"{str(frame_id).zfill(5)}.jpg")
#     frame = cv2.imread(frame_path)
#     if frame is not None:
#         prev_positions = draw_boxes_on_frame(frame, detections_by_frame[frame_id], prev_positions, frame_id)
#         out.write(frame)
# 
# out.release()
# cv2.destroyAllWindows()